# Agent配置
## Flexible-Agent
本节目的让Agent真正具备智能与个性，就必须为它设定：
- 它该如何思考（instructions）
- 它依赖的模型是什么（model）
- 它能使用哪些外部工具（tools）

配置 = 定义 Agent 的“性格 + 能力 + 工具箱”。

## 核心配置项

| 配置项         | 说明                                                                 |
| -------------- | -------------------------------------------------------------------- |
| name           | Agent 的名称，用于标识与管理                                         |
| instructions   | 系统提示（System Prompt），定义 Agent 的角色与行为风格                |
| model          | 使用的模型（如 gpt-4o-mini, gpt-4-turbo, gpt-3.5-turbo 等）          |
| tools          | Agent 可调用的工具（如浏览器、Python 代码执行器、文件处理工具）       |
| model_settings | 对模型的参数微调，如温度 temperature、采样率 top_p。                  |
| temperature    | 控制生成的随机性，影响创造性与稳定性                                 |

## 示例拆解--工具使用
天气助手

### 导入pip

```python
from agents import Agent, ModelSettings, function_tool, Runner, set_tracing_disabled
from agents.extensions.models.litellm_model import LitellmModel
import os, asyncio
from dotenv import load_dotenv
```

- Agent：定义智能体
- function_tool：用于装饰自定义函数，让 Agent 能调用
- Runner：执行 Agent
- LitellmModel：包装对 Mistral API 的访问

### 环境与模型配置

```python
load_dotenv()
api_key = os.getenv('mistral_key')
base_url = 'https://api.mistral.ai/v1'
chat_model = "mistral/mistral-small-latest"
set_tracing_disabled(disabled=True)
llm = LitellmModel(model=chat_model, api_key=api_key, base_url=base_url)
```

- 从 .env 文件加载 Mistral API Key
- 设置模型为 Mistral Small Latest
- LitellmModel 封装了请求接口
- 关闭追踪（方便本地运行）

### 定义工具函数

```python
@function_tool 
def get_weather(city: str) -> str: 
    """查询一个城市的天气
    Args:
        city (str): 要查询的城市名称
    Returns:
        str: 城市的天气情况
    """
    print(f"天气查询工具被调用了，查询的是{city}")
    return f"The weather in {city} is sunny" 
```

- @function_tool 会把这个函数注册为 Agent 的一个“外部工具”
- 函数的参数和返回值会被自动转换为 JSON Schema，用于模型理解和调用

### 创建Agent

```python
agent = Agent(
    name="天气助手",
    instructions="你是一个专业的天气助手，能够查询不同城市的天气情况。",
    model=llm,
    tools=[get_weather]
)
```

- name：Agent 名称；
- instructions：告诉它回答风格；
- tools：绑定刚定义的 get_weather；
- model：绑定前面构造的 LLM。

### 运行Agent

```python
async def main():
    result = await Runner.run(agent, "上海最近适合出去玩吗?")
    print(result.final_output)
```

## 输出类型（output_type）
默认情况下，Agent输出是字符串类型`str`
但是大部分情况下，我们需要的是结构化数据，如JSON、字典、表格等。

示例：Pydantic对象输出

```python
class Candidate(BaseModel):
    name: str
    gender: str
    location: str
# 定义一个结构化类型
agent = Agent(
    name="简历助手"
    instructions="根据要求的格式提取相应的信息"
    model=llm,
    output_type=Candidate
)
# 执行层
result = await Runner.run(agent,"我叫李婷，女，现居深圳，需要应聘项目经理岗位。")
print(result.final_output)
```

这是一个Pydantic对象，方便后续处理或保存数据库

### 上下文（Context）

LLM本身无状态，通过与设置上下文（Context）来实现状态管理，包含：身份定义、临时变量设定等

上下文是通过RunContextWrapper封装传递给Agent、工具等对象。
其核心作用是：
> "为每次Agent运行提供共享的依赖和状态"

示例：
```python
@dataclass
class UseInfo:
    name:str
    uid:int

# 定义上下文类

@function_tool
async def fetch_user_age(wrapper:RunContextWrapper[UseInfo]) -> int:
    return f"用户{wrapper.context.name}的年龄是"
```
- 工具函数通过wrapper.context访问当前用户
- 不同Agent、工具之间共享相同的上下文类系

```python
result = await Runner.run(
    starting_agent=agent,
    input="What is the age of the user?",
    context=user_info,
)
```
- context=user_info ：将user_info传递给Agent，传递上下文作用
- 输出：The user John is 47 years old.

>上下文不会发送给LLM，而是本地的依赖容器。